# **I) Python et Data Engineering**

**Objectif** : Réaliser un code clair et proprement structuré. Mettre en avant les éléments considérés comme essentiels pour du code utilisable dans un environnement de production. Mettre l’accent sur vos connaissances en conception de jobs de manipulation de données ainsi que les bonnes pratiques python.

1. Les données

Vous avez à votre disposition les 4 fichiers de données suivants :
- **drugs.csv** : contient les noms de drugs (des médicaments) avec un id (atccode) et un nom (drug)
- **pubmed.csv** : contient des titres d’articles PubMed (title) associés à un journal (journal) à une date donnée (date) ainsi qu’un id (id)
- **pubmed.json** : même structure que pubmed.csv mais en format JSON
- **clinical_trials.csv** : contient des publications scientifiques avec un titre (scientific_title), un id (id), un journal (journal) et une date (date).

In [1]:
import pandas as pd

drugs = pd.read_csv('drugs.csv')
pubmed = pd.read_csv('pubmed.csv')
trials = pd.read_csv('clinical_trials.csv', names=['id', 'title', 'date', 'journal'], header = 0, converters={'title': str.strip})

# 2. Le travail à réaliser
L’objectif est de construire une data pipeline permettant de traiter les données définies dans la partie précédente afin de générer le résultat décrit dans la partie 3.
Pour ce faire, vous devez mettre en place un projet en python organisé de la manière qui vous paraît la plus pertinente pour résoudre ce problème. Nous attendons que vous identifiiez une structure de projet et une séparation des étapes nécessaires qui permettent de mettre en évidence vos connaissances autour du développement de jobs data en python.

Il faudra essayer de considérer les hypothèses de travail suivantes :
- Certaines étapes de votre data pipeline pourraient être réutilisées par d’autres data pipelines
- On suppose que votre travail devra être intégré dans un orchestrateur de jobs (de type DAG) par la suite, votre code et la structure choisie doivent donc favoriser cette intégration
- Votre code doit respecter les pratiques que vous mettriez en place dans un cadre professionnel au sein d’une équipe de plusieurs personnes

Nous laissons volontairement un cadre assez libre pour voir votre manière de structurer un projet, de rédiger votre code et de mettre en place les éléments qui vous semblent essentiels dans un projet d’équipe. N’hésitez pas à argumenter votre proposition et les choix que vous faites si nécessaire.

In [2]:
def removeEmptyRows(df):
    liste = []
    for i, row in df.iterrows():
        if row.title == '' or row.title is None:
            liste.append(i)
    
    df = df.drop(liste)
    return df

In [3]:
trials = removeEmptyRows(trials)

In [4]:
def findEmptyValueDF(df, key):
    for i, row in df.iterrows():
        data = df[df[key] == row[key]]
        
        if data.shape[0] > 1:
            for j, val in row.items():
                if val != val:
                    df[j][i] = data[j][data.index != i].values[0]

    return df.drop_duplicates()

In [5]:
trials = findEmptyValueDF(trials, 'title')

# 3. Data pipeline

Votre data pipeline doit produire en sortie un fichier JSON qui représente un graphe de liaison entre les différents médicaments et leurs mentions respectives dans les différentes publications PubMed, les différentes publications scientifiques et enfin les journaux avec la date associée à chacune de ces mentions. La représentation ci-dessous permet de visualiser ce qui est attendu. Il peut y avoir plusieurs manières de modéliser cet output et vous pouvez justifier votre vision :

Règles de gestion :
- Un drug est considéré comme mentionné dans un article PubMed ou un essai clinique s’il est mentionné dans le titre de la publication.
- Un drug est considéré comme mentionné par un journal s’il est mentionné dans une publication émise par ce journal.

In [7]:
source = drugs.drug
source

0    DIPHENHYDRAMINE
1       TETRACYCLINE
2            ETHANOL
3           ATROPINE
4        EPINEPHRINE
5       ISOPRENALINE
6      BETAMETHASONE
Name: drug, dtype: object

In [8]:
pubmed['source'] = 'pubmed'

In [9]:
trials['source'] = 'trials'

In [10]:
target = pubmed.append(trials)
target

,id,title,date,journal,source
0,1,A 44-year-old man with erythema of the face di...,01/01/2019,Journal of emergency nursing,pubmed
1,2,"An evaluation of benadryl, pyribenzamine, and ...",01/01/2019,Journal of emergency nursing,pubmed
2,3,Diphenhydramine hydrochloride helps symptoms o...,02/01/2019,The Journal of pediatrics,pubmed
3,4,Tetracycline Resistance Patterns of Lactobacil...,01/01/2020,Journal of food protection,pubmed
4,5,Appositional Tetracycline bone formation rates...,02/01/2020,American journal of veterinary research,pubmed
5,6,Rapid reacquisition of contextual fear followi...,2020-01-01,Psychopharmacology,pubmed
6,7,The High Cost of Epinephrine Autoinjectors and...,01/02/2020,The journal of allergy and clinical immunology...,pubmed
7,8,Time to epinephrine treatment is associated wi...,01/03/2020,The journal of allergy and clinical immunology...,pubmed
0,NCT01967433,Use of Diphenhydramine as an Adjunctive Sedati...,1 January 2020,Journal of emergency nursing,trials
1,NCT04189588,Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydroch...,1 January 2020,Journal of emergency nursing,trials


In [11]:
def listLiaison(source, target):
    from re import search

    liste = []

    for i in source:
        for j, row in target.iterrows():
            match = search(i.upper(), row.title.upper())

            if match:
                liste.append({'drug' : i, 'id' : row.id, 'title' : row.title, 'date' : row.date, 'journal' : row.journal, 'source' : row.source})
    return liste

In [12]:
liste_match = listLiaison(source, target)

In [13]:
liste_publication = liste_match[liste_match]

TypeError: list indices must be integers or slices, not list

In [ ]:
import json

with open('data.json', 'w') as f:
    json.dump(liste_match, f)

# 4. Traitement ad-hoc

Vous devez aussi mettre en place (hors de la data pipeline, vous pouvez considérer que c’est une partie annexe) une feature permettant de répondre à la problématique suivante :
- Extraire depuis le json produit par la data pipeline le nom du journal qui mentionne le plus de médicaments différents ?

In [ ]:
data = pd.read_json('data.json')

In [ ]:
data_count = data.groupby('journal').drug.nunique()

In [ ]:
df_grouped = pd.DataFrame(dict(journal = data_count.index, nombre = data_count.values))
df_grouped.sort_values(by=['nombre'], ascending=False)